In [ ]:
%cd /content

In [ ]:
import os
if os.path.exists('/content/hiwyn'):
  %rm -rf /content/hiwyn
!git clone https://github.com/ByeongHyunPak/hiwyn.git
%cd /content/hiwyn

In [ ]:
!sudo apt-get install ninja-build
!ninja --version

In [ ]:
if os.path.exists('/content/hiwyn/nvdiffrast'):
  %rm -rf /content/hiwyn/nvdiffrast
!git clone --recursive https://github.com/NVlabs/nvdiffrast
%cd /content/hiwyn/nvdiffrast
!pip install .
%cd /content/hiwyn

In [ ]:
import cv2

import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

import nvdiffrast.torch as dr

from tqdm import tqdm
from IPython.display import Image
from torchvision.transforms import ToPILImage, ToTensor
from einops import rearrange, reduce, repeat

In [ ]:
use_opengl = False # On T4 GPU, only False works, but rasterizer works much better if = True
glctx = dr.RasterizeGLContext() if use_opengl else dr.RasterizeCudaContext()

In [ ]:
from multidiffusion import MultiDiffusion
from utils import cond_noise_sampling, identity_latent_warping, compute_erp_up_noise_pred
to_img = ToPILImage()

In [ ]:
# Sample the source noise, channel = 3
up_level = 3   # Upsampling level k
batch_size = 1 # Batch size
dim_channel = 3    # Channel dimension
H = 64   # Original H
W = 64   # Original W

src_noise = torch.randn(batch_size,dim_channel,H,W)

# Upscale to 512 x 512 and visualize, just for visualization purposes
view_test_noise = F.interpolate(src_noise, size=(512,512), mode='nearest')
to_img(view_test_noise[0])

In [ ]:
# Generate conditionally upsampled noise by k = up_level
up_noise = cond_noise_sampling(src_noise, level=up_level)


# Visualize upsampled noise
test_upsampled_noise_vis = F.interpolate(up_noise, size=(512,512), mode='nearest')
to_img(test_upsampled_noise_vis[0])

In [ ]:
tgt_noise = identity_latent_warping(up_noise, (H, W), glctx)

In [ ]:
up_vis_fast = F.interpolate(tgt_noise, size=(512,512), mode='nearest')
to_img(up_vis_fast[0])